In [1]:
import os
import sys
# 将项目根目录（MyHub）的父目录添加到系统路径中
# 这使得 'MyHub' 本身可以被当作一个包来导入
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import json
import torch
import time
import argparse
import datetime
import numpy as np
from pathlib import Path
import timm.optim.optim_factory as optim_factory
from torch.utils.tensorboard.writer import SummaryWriter

from MyHub.core.registry import DATASETS, MODELS, POSTFUNCS, TRANSFORMS, EVALUATORS, build_from_registry
from MyHub.training_scripts.utils import misc
from MyHub.training_scripts.utils.yaml import load_yaml_config,split_config, add_attr
from MyHub.training_scripts.trainer import train_one_epoch
from MyHub.training_scripts.tester import test_one_epoch

/home/chengxiaozhen/anaconda3/envs/forensichub2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.2
Current Torch version: 2.1


In [2]:
parser = argparse.ArgumentParser(description='Training Script')
parser.add_argument('--config', default='config/resnet_train.yaml', help='path to config file', type=str)

args = parser.parse_args(['--config', '/home/chengxiaozhen/Test/ForensicHub/MyHub/config/resnet_train.yaml'])

config = load_yaml_config(args.config)
args, model_args, train_dataset_args, test_dataset_args, transform_args, evaluator_args = split_config(config)
add_attr(args,output_dir=args.log_dir)
add_attr(args,if_not_amp=not args.use_amp)

print(args)
print(model_args)
print(train_dataset_args)
print(test_dataset_args)
print(transform_args)

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir, exist_ok=True)

Namespace(gpus='0,1', flag='train', log_dir='./log/resnet_train', if_predict_label=True, if_predict_mask=False, batch_size=2, test_batch_size=128, epochs=20, accum_iter=1, record_epoch=20, no_model_eval=False, test_period=1, log_per_epoch_count=20, find_unused_parameters=True, use_amp=False, weight_decay=0.05, lr=0.0001, blr=0.001, min_lr=1e-05, warmup_epochs=1, device='cuda', seed=42, resume='', start_epoch=0, num_workers=8, pin_mem=True, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', output_dir='./log/resnet_train', if_not_amp=True)
{'name': 'Resnet101', 'init_config': {'output_type': 'label', 'image_size': 256}}
{'name': 'CrossDataset', 'dataset_name': 'CrossDataset', 'init_config': {'dataset_config': [{'name': 'LabelDataset', 'pic_nums': 12641, 'init_config': {'image_size': 256, 'path': '/home/chengxiaozhen/Test/ForensicHub/MyHub/data/train_data/CASIAv2.json'}}]}}
[{'name': 'LabelDataset', 'dataset_name': 'Coverage', 'init_config': {'image_size': 256, 'path': '/

In [3]:
transform = build_from_registry(TRANSFORMS, transform_args)
train_transform = transform.get_train_transform()
test_transform = transform.get_test_transform()
post_transform = transform.get_post_transform()
print("\nTrain transform: ", train_transform)
print("\nTest transform: ", test_transform)
print("\nPost transform: ", post_transform)


[Lazy import] 从MyHub.common.transforms.CrossTransform 加载 CrossTransform
[build_from_registry] 创建模型 'CrossTransform' 参数: {}

Train transform:  Compose([
  HorizontalFlip(always_apply=False, p=0.5),
  VerticalFlip(always_apply=False, p=0.5),
  RandomBrightnessContrast(always_apply=False, p=1, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), brightness_by_max=True),
  ImageCompression(always_apply=False, p=0.2, quality_lower=70, quality_upper=100, compression_type=0),
  RandomRotate90(always_apply=False, p=0.5),
  GaussianBlur(always_apply=False, p=0.2, blur_limit=(3, 7), sigma_limit=(0, 0)),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})

Test transform:  Compose([
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})

Post transform:  Compose([
  Normalize(always_apply=False, p=1.0, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=True),
], p=1.0, bbo

In [4]:
# get post function (if have)
post_function_name = f"{model_args['name']}_post_func".lower()
if model_args.get('post_func_name') is not None:
    post_function_name = f"{model_args['post_func_name']}_post_func".lower()
print(f"Post function check: {post_function_name}")
if POSTFUNCS.has(post_function_name):
    post_function = POSTFUNCS.get(post_function_name)
else:
    post_function = None
print(post_function)

Post function check: resnet101_post_func
None


In [5]:
train_dataset_args["init_config"].update({
        "post_funcs": post_function,
        "common_transform": train_transform,
        "post_transform": post_transform
    })
train_dataset = build_from_registry(DATASETS, train_dataset_args)

[Lazy import] 从MyHub.common.datasets.CrossDataset 加载 CrossDataset
[build_from_registry] 创建模型 'CrossDataset' 参数: {'dataset_config': [{'name': 'LabelDataset', 'pic_nums': 12641, 'init_config': {'image_size': 256, 'path': '/home/chengxiaozhen/Test/ForensicHub/MyHub/data/train_data/CASIAv2.json'}}], 'post_funcs': None, 'common_transform': Compose([
  HorizontalFlip(always_apply=False, p=0.5),
  VerticalFlip(always_apply=False, p=0.5),
  RandomBrightnessContrast(always_apply=False, p=1, brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), brightness_by_max=True),
  ImageCompression(always_apply=False, p=0.2, quality_lower=70, quality_upper=100, compression_type=0),
  RandomRotate90(always_apply=False, p=0.5),
  GaussianBlur(always_apply=False, p=0.2, blur_limit=(3, 7), sigma_limit=(0, 0)),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}), 'post_transform': Compose([
  Normalize(always_apply=False, p=1.0, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max

In [6]:
test_dataset_list = {}
for test_args in test_dataset_args:
    test_args["init_config"].update({
        "post_funcs": post_function,
        "common_transform": test_transform,
        "post_transform": post_transform
    })
    test_dataset_list[test_args["dataset_name"]] = build_from_registry(DATASETS, test_args)

[build_from_registry] 创建模型 'LabelDataset' 参数: {'image_size': 256, 'path': '/home/chengxiaozhen/Test/ForensicHub/MyHub/data/test_data/coverage.json', 'is_resizing': True, 'is_padding': False, 'post_funcs': None, 'common_transform': Compose([
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}), 'post_transform': Compose([
  Normalize(always_apply=False, p=1.0, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=True),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})}


In [7]:
print(f"Train dataset: {train_dataset_args['dataset_name']}.")
print(len(train_dataset))
print(str(train_dataset))
print(f"Test dataset: {[args['dataset_name'] for args in test_dataset_args]}.")
print([len(dataset) for dataset in test_dataset_list.values()])

Train dataset: CrossDataset.
12641
<=== CrossDataset with 1 datasets: ['LabelDataset'] ===>

[Dataset 0 - LabelDataset]
LabelDataset from /home/chengxiaozhen/Test/ForensicHub/MyHub/data/train_data/CASIAv2.json
Total samples: 12614
Label 0 samples (real): 7491
Label 1 samples (fake): 5123

Total samples per epoch: 12,641
<================================================>

Test dataset: ['Coverage'].
[200]


In [8]:
test_sampler = {}
sampler_train = torch.utils.data.RandomSampler(train_dataset)
for test_dataset_name, dataset_test in test_dataset_list.items():
    sampler_test = torch.utils.data.RandomSampler(dataset_test)
    test_sampler[test_dataset_name] = sampler_test

In [9]:
global_rank = 0
if global_rank == 0 and args.log_dir is not None:
    os.makedirs(args.log_dir, exist_ok=True)
    log_writer = SummaryWriter(log_dir=args.log_dir)

In [10]:
data_loader_train = torch.utils.data.DataLoader(
    train_dataset, 
    sampler=sampler_train,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=True,
)

In [11]:
test_dataloaders = {}
for test_dataset_name in test_sampler.keys():
    test_dataloader = torch.utils.data.DataLoader(
        test_dataset_list[test_dataset_name],
        sampler=test_sampler[test_dataset_name],
        batch_size=args.test_batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )
    test_dataloaders[test_dataset_name] = test_dataloader

In [12]:
model = build_from_registry(MODELS, model_args)

[Lazy import] 从MyHub.common.backbones.resnet 加载 Resnet101
[build_from_registry] 创建模型 'Resnet101' 参数: {'output_type': 'label', 'image_size': 256}


In [13]:
evaluator_list = []
for eva_args in evaluator_args:
    evaluator_list.append(build_from_registry(EVALUATORS, eva_args))
print(f"Evaluators: {evaluator_list}")

[Lazy import] 从MyHub.common.evaluations.F1 加载 ImageF1
[build_from_registry] 创建模型 'ImageF1' 参数: {'threshold': 0.5}
Evaluators: [<MyHub.common.evaluations.F1.ImageF1 object at 0x7fca58f91cc0>]


In [14]:
device = torch.device(args.device)
model = model.to(device)

In [15]:
eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()
if args.lr is None:  # only base_lr is specified
    args.lr = args.blr * eff_batch_size / 256

print("base lr: %.2e" % (args.lr * 256 / eff_batch_size))
print("actual lr: %.2e" % args.lr)
print("accumulate grad iterations: %d" % args.accum_iter)
print("effective batch size: %d" % eff_batch_size)

base lr: 1.28e-02
actual lr: 1.00e-04
accumulate grad iterations: 1
effective batch size: 2


In [16]:
args.opt = 'AdamW'
args.betas = (0.9, 0.999)
args.momentum = 0.9
optimizer = optim_factory.create_optimizer(args, model)
print(optimizer)
loss_scaler = misc.NativeScalerWithGradNormCount()

misc.load_model(args=args, model_without_ddp=model, optimizer=optimizer, loss_scaler=loss_scaler)

print(f"Start training for {args.epochs} epochs")

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0.05
)
Start training for 20 epochs


In [17]:
start_time = time.time()
best_evaluate_metric_value = 0
for epoch in range(args.start_epoch, args.epochs):
    # train for one epoch
    train_stats = train_one_epoch(
        model, 
        data_loader_train,
        optimizer, 
        device, 
        epoch, 
        loss_scaler,
        log_writer=log_writer,
        log_per_epoch_count=args.log_per_epoch_count,
        args=args
    )
    print(f"Epoch {epoch} training stats: {train_stats}")
    break
    

Auto Mixed Precision (AMP) is disabled.
log_dir: ./log/resnet_train
Epoch: [0]  [   0/6320]  eta: 1:38:18  lr: 0.000000  combined_loss: [local: 0.6877 | reduced: 0.6877]  time: 0.9333  data: 0.2728  max mem: 841
Epoch: [0]  [  20/6320]  eta: 0:10:52  lr: 0.000000  combined_loss: [local: 0.6749 | reduced: 0.6781]  time: 0.0621  data: 0.0001  max mem: 1017
Epoch: [0]  [  40/6320]  eta: 0:08:44  lr: 0.000001  combined_loss: [local: 0.6899 | reduced: 0.6824]  time: 0.0623  data: 0.0001  max mem: 1017
Epoch: [0]  [  60/6320]  eta: 0:07:59  lr: 0.000001  combined_loss: [local: 0.6884 | reduced: 0.6827]  time: 0.0626  data: 0.0001  max mem: 1017
Epoch: [0]  [  80/6320]  eta: 0:07:36  lr: 0.000001  combined_loss: [local: 0.6859 | reduced: 0.6834]  time: 0.0627  data: 0.0001  max mem: 1017
Epoch: [0]  [ 100/6320]  eta: 0:07:22  lr: 0.000002  combined_loss: [local: 0.6809 | reduced: 0.6838]  time: 0.0628  data: 0.0001  max mem: 1017
Epoch: [0]  [ 120/6320]  eta: 0:07:12  lr: 0.000002  combined_l